# 文本分割策略深入探討

本範例展示：
1. **第1個儲存格**：5種文本分割策略並建立向量資料庫
2. **第2個儲存格**：比較不同分割策略的查詢結果

學習目標：理解文本分割對檢索效果的影響，選擇最適合的分割策略

In [ ]:
# 第1個儲存格：5種文本分割策略並建立向量資料庫

import os
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TextSplitter,
    TokenTextSplitter,
)
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義包含文字檔案的目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "信用卡權益說明.txt")
db_dir = os.path.join(current_dir, "db")

# 檢查文字檔案是否存在
if not os.path.exists(file_path):
    raise FileNotFoundError(
        f"檔案 {file_path} 不存在。請檢查路徑。"
    )

# 從檔案讀取文字內容
loader = TextLoader(file_path)
documents = loader.load()

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-zh"
)

# 建立並持久化向量存儲的函數
def create_vector_store(docs, store_name):
    persistent_directory = os.path.join(db_dir, store_name)
    if not os.path.exists(persistent_directory):
        print(f"\n--- 正在建立向量存儲 {store_name} ---")
        db = Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory
        )
        print(f"--- 完成建立向量存儲 {store_name} ---")
    else:
        print(
            f"向量存儲 {store_name} 已存在。無需初始化。")

# 1. 基於字符的分割
# 根據指定的字符數將文本分割成塊
# 適用於無論內容結構如何都需要一致塊大小的情況
print("\n--- 使用基於字符的分割 ---")
char_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
char_docs = char_splitter.split_documents(documents)
create_vector_store(char_docs, "chroma_db_char_nb")

# 2. 基於句子的分割
# 根據句子將文本分割成塊，確保塊在句子邊界處結束
# 適用於在塊內保持語義連貫性
print("\n--- 使用基於句子的分割 ---")
sent_splitter = SentenceTransformersTokenTextSplitter(chunk_size=1000)
sent_docs = sent_splitter.split_documents(documents)
create_vector_store(sent_docs, "chroma_db_sent_nb")

# 3. 基於標記的分割
# 使用標記器（如 GPT-2）根據標記（單詞或子詞）將文本分割成塊
# 適用於具有嚴格標記限制的轉換器模型
print("\n--- 使用基於標記的分割 ---")
token_splitter = TokenTextSplitter(chunk_overlap=0, chunk_size=512)
token_docs = token_splitter.split_documents(documents)
create_vector_store(token_docs, "chroma_db_token_nb")

# 4. 遞迴基於字符的分割
# 嘗試在字符限制內的自然邊界（句子、段落）處分割文本
# 在保持連貫性和遵守字符限制之間取得平衡
print("\n--- 使用遞迴基於字符的分割 ---")
rec_char_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100)
rec_char_docs = rec_char_splitter.split_documents(documents)
create_vector_store(rec_char_docs, "chroma_db_rec_char_nb")

# 5. 自訂分割
# 允許根據特定需求建立自訂分割邏輯
# 適用於標準分割器無法處理的具有獨特結構的文件
print("\n--- 使用自訂分割 ---")

class CustomTextSplitter(TextSplitter):
    def split_text(self, text):
        # 自訂分割文本的邏輯
        return text.split("\n\n")  # 範例：按段落分割

custom_splitter = CustomTextSplitter()
custom_docs = custom_splitter.split_documents(documents)
create_vector_store(custom_docs, "chroma_db_custom_nb")

print("\n=== 所有向量存儲已建立完成 ===")

In [ ]:
# 第3個儲存格：整合 Chain 功能 - 比較不同分割策略的 RAG Chain

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 定義路徑
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 定義 Prompt 模板
prompt = ChatPromptTemplate.from_template("""你是一位專業的客服助理，請根據提供的資料回答使用者問題。

參考資料：
{context}

問題：{question}

請提供清楚、準確的回答：""")

# 格式化文件的函數
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 使用不同分割策略的 RAG Chain 並比較結果
def compare_rag_chains(store_name, query, strategy_name):
    persistent_directory = os.path.join(db_dir, store_name)
    if os.path.exists(persistent_directory):
        print(f"\n{'='*70}")
        print(f"分割策略：{strategy_name}")
        print(f"{'='*70}")
        
        # 載入向量資料庫
        db = Chroma(
            persist_directory=persistent_directory,
            embedding_function=embeddings
        )
        
        # 建立檢索器
        retriever = db.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 2}
        )
        
        # 步驟 1：檢索相關文件
        print(f"\n📋 步驟 1：檢索相關文件")
        retrieved_docs = retriever.invoke(query)
        print(f"找到 {len(retrieved_docs)} 個相關文件")
        for i, doc in enumerate(retrieved_docs, 1):
            print(f"\n文件 {i} 內容預覽：")
            print(f"{doc.page_content[:150]}...")
        
        # 步驟 2：建立 RAG Chain
        print(f"\n⛓️  步驟 2：建立 RAG Chain")
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )
        
        # 步驟 3：執行 Chain 並取得答案
        print(f"\n🤖 步驟 3：生成回答")
        answer = rag_chain.invoke(query)
        print(f"\n【{strategy_name} 的回答】")
        print(answer)
        print(f"\n{'-'*70}")
        
        return answer
    else:
        print(f"向量存儲 {store_name} 不存在。")
        return None

# 定義使用者的問題
query = "信用卡的現金回饋比例是多少？年費如何計算？"

print("\n" + "="*70)
print(f"查詢問題：{query}")
print("="*70)

# 比較不同分割策略的 RAG Chain 效果
results = {}
results["CharacterTextSplitter"] = compare_rag_chains(
    "chroma_db_char_nb", query, "CharacterTextSplitter (字符分割)"
)
results["SentenceTransformersTokenTextSplitter"] = compare_rag_chains(
    "chroma_db_sent_nb", query, "SentenceTransformersTokenTextSplitter (句子分割)"
)
results["RecursiveCharacterTextSplitter"] = compare_rag_chains(
    "chroma_db_rec_char_nb", query, "RecursiveCharacterTextSplitter (遞迴字符分割)"
)

print("\n" + "="*70)
print("📊 分割策略對 RAG Chain 的影響總結")
print("="*70)
print("""
🔍 觀察重點：
1. 不同分割策略檢索到的文件內容是否完整
2. 分割邊界是否切斷了重要資訊
3. 哪種策略的答案最準確、最完整

💡 最佳實踐建議：
- CharacterTextSplitter：適用於結構簡單、格式統一的文件
- SentenceTransformers：適合需要保持語義完整性的文件
- RecursiveCharacterTextSplitter：推薦！在完整性和靈活性之間取得最佳平衡

📌 關鍵學習：
文本分割策略直接影響 RAG 系統的回答品質。
選擇合適的分割策略能夠：
✅ 避免重要資訊被切斷
✅ 提高檢索的準確性
✅ 改善最終答案的完整度
""")
print("="*70)